# Handling Missing Values With Scikit Learn
---

Using the same loan dataframe from the previous notes and the link to the kaggle dataset is [here](https://www.kaggle.com/datasets/tanishaj225/loancsv)

In [1]:
import kagglehub
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from kagglehub import KaggleDatasetAdapter
from sklearn.impute import SimpleImputer
import warnings

# Ignore only FutureWarning and DeprecationWarning
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
# Set the path to the file you'd like to load
file_path = "loan.csv"  # file inside tanishaj225/loancsv dataset

# Load dataset into pandas DataFrame
dataset = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "tanishaj225/loancsv",   # dataset slug
    path=file_path           # specify file to load
)

In [3]:
dataset.head(10)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
5,LP001011,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y
6,LP001013,Male,Yes,0,Not Graduate,No,2333,1516.0,95.0,360.0,1.0,Urban,Y
7,LP001014,Male,Yes,3+,Graduate,No,3036,2504.0,158.0,360.0,0.0,Semiurban,N
8,LP001018,Male,Yes,2,Graduate,No,4006,1526.0,168.0,360.0,1.0,Urban,Y
9,LP001020,Male,Yes,1,Graduate,No,12841,10968.0,349.0,360.0,1.0,Semiurban,N


In [4]:
dataset.isna().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [5]:
dataset.select_dtypes(include='float64').columns

Index(['CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term',
       'Credit_History'],
      dtype='object')

#### SimpleImputer provides several hyperparameters that let you control how missing values are handled. Some of the key ones are:

1. `missing_values = NaN` → Specifies which value should be considered as missing (by default, it looks for `NaN`).

2. `strategy = 'mean'` → Defines the method used to replace missing values. Options include `'mean'`, `'median'`, `'most_frequent'`, or `'constant'`.

3. `fill_value = None` → The value used when `strategy = 'constant'`. For example, you could fill all missing entries with `0` or `"Unknown"`.

4. `copy = True` → Ensures the original dataset is not changed; instead, a copy is modified.

5. `add_indicator = False` → If set to `True`, an extra column is added that indicates whether a value was missing or not.

6. `keep_empty_features = False` → Controls whether to keep features (columns) that only contain missing values. By default, such columns are dropped.

---

Since we are dealing with a numerical data I will be using `strategy='mean'`

In [6]:
si = SimpleImputer(strategy='mean')
si.fit_transform(dataset[dataset.select_dtypes(include='float64').columns])

array([[0.00000000e+00, 1.46412162e+02, 3.60000000e+02, 1.00000000e+00],
       [1.50800000e+03, 1.28000000e+02, 3.60000000e+02, 1.00000000e+00],
       [0.00000000e+00, 6.60000000e+01, 3.60000000e+02, 1.00000000e+00],
       ...,
       [2.40000000e+02, 2.53000000e+02, 3.60000000e+02, 1.00000000e+00],
       [0.00000000e+00, 1.87000000e+02, 3.60000000e+02, 1.00000000e+00],
       [0.00000000e+00, 1.33000000e+02, 3.60000000e+02, 0.00000000e+00]],
      shape=(614, 4))

Since the output is in a form of an array so wil save the values inside a variable and then we will upload it to the dataframe like:

In [7]:
ar = si.fit_transform(dataset[dataset.select_dtypes(include='float64').columns])

In [8]:
new_dataset = pd.DataFrame(ar,columns=dataset.select_dtypes(include='float64').columns)

In [9]:
new_dataset.isna().sum()

CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
dtype: int64

In [10]:
new_dataset.head(10)

,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
0,0.0,146.412162,360.0,1.0
1,1508.0,128.000000,360.0,1.0
2,0.0,66.000000,360.0,1.0
3,2358.0,120.000000,360.0,1.0
4,0.0,141.000000,360.0,1.0
5,4196.0,267.000000,360.0,1.0
6,1516.0,95.000000,360.0,1.0
7,2504.0,158.000000,360.0,0.0
8,1526.0,168.000000,360.0,1.0
9,10968.0,349.000000,360.0,1.0


#### Although this solves the problem of missing values but it also bring a new problem i.e. duplicate data

* When you use mean imputation, all missing values in a column are replaced with the exact same value (the mean).

* So if 50 rows were missing, all 50 get the same replacement → this creates many duplicate entries in that column.

* It’s not "wrong," but it reduces variance and can cause bias in ML models.

---

### How to counter this

There are a few strategies depending on how serious the problem is:

1. Median instead of mean

        si = SimpleImputer(strategy='median')

    Median is less sensitive to outliers, so duplicates still appear, but they’re less biased.

2. Use `most_frequent` for categorical-like floats
    If the float column behaves like categories (e.g., ratings 1, 2, 3), you might fill with the most common value.

3. Use interpolation (better for time series / ordered data)

        dataset['col'] = dataset['col'].interpolate(method='linear')

    This fills missing values with values “in between,” reducing duplicates.

4. KNN Imputer (smarter imputation)

        from sklearn.impute import KNNImputer
        knn = KNNImputer(n_neighbors=5)
        new_dataset = pd.DataFrame(
            knn.fit_transform(dataset.select_dtypes(include='float64')),
            columns=dataset.select_dtypes(include='float64').columns
       )

Here, missing values are replaced using similar rows, so you don’t get identical duplicates everywhere.

5. Drop rows if duplicates overwhelm the dataset
If a column is heavily missing (like 40–50%), sometimes dropping that column/row is better.

In [11]:
from sklearn.impute import KNNImputer

In [12]:
knn = KNNImputer(n_neighbors=5)
new_dataset = pd.DataFrame(knn.fit_transform(dataset.select_dtypes(include='float64')), columns=dataset.select_dtypes(include='float64').columns)

In [13]:
new_dataset.head(10)

,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
0,0.0,216.4,360.0,1.0
1,1508.0,128.0,360.0,1.0
2,0.0,66.0,360.0,1.0
3,2358.0,120.0,360.0,1.0
4,0.0,141.0,360.0,1.0
5,4196.0,267.0,360.0,1.0
6,1516.0,95.0,360.0,1.0
7,2504.0,158.0,360.0,0.0
8,1526.0,168.0,360.0,1.0
9,10968.0,349.0,360.0,1.0


---
## SimpleImputer

### Advantages

* **Fast & efficient** → Works instantly even on very large datasets.

* **Easy to implement** → Just one line of code with no complexity.

* **Stable & reproducible** → Always imputes the same value (e.g., mean, median).

* **Good as a baseline** → Quick fix to prepare data for initial models.

* **Less risk of overfitting** → Since it doesn’t “learn” from neighbors.

### Disadvantages

* **Ignores feature relationships** → Just fills a fixed value, doesn’t consider correlations.

* **Introduces bias** → Mean/median replacement can distort the distribution.

* **Reduces variance** → Too many repeated values (e.g., mean everywhere).

* **Not realistic** → Imputed values may not represent real-world behavior.

---

## KNNImputer

### Advantages

* **Preserves feature relationships** → Uses neighboring rows, so correlations are maintained.
 
* **More realistic imputation** → Produces values closer to the actual missing ones.

* **Maintains variance** → Avoids duplicate values like SimpleImputer does.

* **Works well for both** categorical & numerical (after encoding).

* **Improves ML performance** → Models trained on KNN-imputed data often perform better.

### Disadvantages

* **Computationally expensive** → Slow for large datasets, since it calculates distances.

* **Memory-heavy** → Requires storing and searching through many rows.

* **Sensitive to scaling** → Distance-based method, so features must be standardized.

* **Sensitive to outliers** → Outliers can distort neighbor selection.

* **Choice of K is tricky** → Too small = noisy imputation, too large = over-smoothed values.

### Rule of thumb:

* Use `SimpleImputer` for speed, simplicity, and very large datasets.

* Use `KNNImputer` when accuracy and maintaining feature relationships are important, and dataset size is manageable.

---

## Encoding

Encoding means converting categorical/text data into numeric values so that machine learning models can understand it.
Because ML algorithms work with numbers (math, distances, probabilities), they can’t directly process words like `"red"`, `"blue"`, `"dog"`, `"cat"`.

### Why Do We Use Encoding?

1. ML algorithms only understand numbers → they can’t calculate on strings.

2. To preserve meaning → encoding helps us represent categories without losing the relationships.

3. Better performance → properly encoded data improves model accuracy.

### Types of Encoding (Most Common)

1. `Label Encoding`

* Assigns each category a number.

* Example:

        Red → 0
        Blue → 1
        Green → 2


* Problem: Implies an order/priority (0 < 1 < 2) even if there isn’t any.

2. `One-Hot Encoding`

* Creates separate binary columns for each category.

* Example:

        Red   → [1, 0, 0]
        Blue  → [0, 1, 0]
        Green → [0, 0, 1]


* No order issue, but increases dimensions if many categories.

3. `Ordinal Encoding`

* Used when categories have a natural order (e.g., "low", "medium", "high").

* Example:

        Low → 1
        Medium → 2
        High → 3


4. `Target / Mean Encoding (advanced)`

* Replace a category with the average target value for that category.

* Example (predicting salary):

        Job: Engineer → 70k avg salary
        Job: Teacher → 50k avg salary
        Job: Doctor → 100k avg salary

In [14]:
# Set the path to the file you'd like to load
file_path = "loan.csv"  # file inside tanishaj225/loancsv dataset

# Load dataset into pandas DataFrame
ds = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "tanishaj225/loancsv",   # dataset slug
    path=file_path           # specify file to load
)

In [15]:
ds.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


So One-Hot Encoding works like:

For example let's take the gender column from the above dataframe so what One-Hot Encoding does is 
instead of using one column, we create two columns (one for each category) like:

| Person | Gender_Male | Gender_Female |
| ------ | ----        | ------        |
| 1      | 1           | 0             |
| 2      | 0           | 1             |
| 3      | 1           | 0             |

Now:

* If Gender = Male → Male=1, Female=0

* If Gender = Female → Male=0, Female=1

This way, the model doesn’t assume order or hierarchy. Each category just becomes a separate feature.

In [16]:
# pip install --upgrade kagglehub

So let's start encoding but first let's check for missing values in gender and the married columns

In [17]:
print(f"The number of missing values in column Genderis: {ds.Gender.isna().sum()} \nThe number of missing values in column Married is: {ds.Married.isna().sum()}")

The number of missing values in column Genderis: 13 
The number of missing values in column Married is: 3


As we can see there are 13 missing values in column Gender and 3 missing values in column Married.

Let's fill them up using mode.

In [18]:
ds.Gender.fillna(ds.Gender.mode()[0],inplace=True)
ds.Married.fillna(ds.Married.mode()[0],inplace=True)

In [19]:
print('After applying the mode method we got')
print(f"The number of missing values in column Genderis: {ds.Gender.isna().sum()} \nThe number of missing values in column Married is: {ds.Married.isna().sum()}")

After applying the mode method we got
The number of missing values in column Genderis: 0 
The number of missing values in column Married is: 0


Now since there is no null values in both the columns let's use Encoding.

There are two ways of applying One-Hot Encoding:

1. Use `pd.get_dummies()` for quick prototyping or EDA.
     * Easiest, quick method when working with DataFrames.
     * `Pros`: Quick, simple, great for small datasets.
     * `Cons`: Limited flexibility (no control over unseen categories in test data).   

2. Use `OneHotEncoder` for production ML models & pipelines.
    * It is class present inside the scikit-learn library
    * More powerful, especially for ML pipelines.
    * `Pros`:
      * Works smoothly inside Scikit-Learn pipelines.
      * Can handle unseen categories (handle_unknown='ignore').
    * `Cons`: Slightly more code compared to get_dummies().

---

### What is a Pipeline?

In ML, a pipeline is like an assembly line in a factory.

Instead of manually applying preprocessing → encoding → scaling → training each time, a pipeline chains them together into one process.

Think of it like:

`Raw Data  →  Preprocessing  →  Feature Encoding  →  Scaling  →  Model Training  →  Predictions`

#### A Pipeline object makes sure:

* Every step runs in the right order.

* Same transformations applied on training data are applied on test data.

* Less risk of data leakage.

* Cleaner code.

### Why Pipelines are Important?

* Prevents **data leakage** (train/test preprocessing mismatch).

* Code is **shorter and cleaner**.

* Easy to reuse (you can save the whole pipeline and deploy it).

* Handles **complex datasets** with mixed feature types easily.

In [20]:
en_data = ds[['Gender','Married']]
en_data.head()

,Gender,Married
0,Male,No
1,Male,Yes
2,Male,Yes
3,Male,Yes
4,Male,No


In [21]:
encoding = pd.get_dummies(en_data)

In [22]:
encoding

,Gender_Female,Gender_Male,Married_No,Married_Yes
0,False,True,True,False
1,False,True,False,True
2,False,True,False,True
3,False,True,False,True
4,False,True,True,False
...,...,...,...,...
609,True,False,True,False
610,False,True,False,True
611,False,True,False,True
612,False,True,False,True


In [23]:
encoding.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   Gender_Female  614 non-null    bool 
 1   Gender_Male    614 non-null    bool 
 2   Married_No     614 non-null    bool 
 3   Married_Yes    614 non-null    bool 
dtypes: bool(4)
memory usage: 2.5 KB


Since machine learning models can only work with numbers, we cannot directly feed them categorical values like 'Male' or 'Female'. These values need to be converted into a numerical representation so that the model can process and learn patterns from them without misinterpreting the categories as having any mathematical order.

So instead of using `pd.get_dummies` we could use `OneHotEncoder` class from scikit-learn

In [24]:
from sklearn.preprocessing import OneHotEncoder

In [25]:
ohe = OneHotEncoder()
ohe.fit_transform(en_data)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1228 stored elements and shape (614, 4)>

#### .fit()

* This step learns the categories from your data.

* For example, if en_data is:

        ["Male", "Female", "Male"]


* then .fit() will figure out that the categories are:

        ['Female', 'Male']

(sklearn usually sorts them alphabetically).

#### .transform()

* Once categories are learned, .transform() converts each row into the one-hot encoded format.

* So the above data becomes:

        Male   → [0, 1]  
        Female → [1, 0]  
        Male   → [0, 1]

#### .fit_transform()

* This is just a shortcut → runs .fit() first (learn categories) then immediately .transform() (encode them).

* So:

        ohe.fit(en_data)        # Step 1: Learn categories
        ohe.transform(en_data)  # Step 2: Encode


* is the same as:

        ohe.fit_transform(en_data)

### Important note:

* You usually call `.fit_transform()` on the training data.

* Then, when you have new data (like test data), you only use `.transform()` (not .fit()) so that it uses the same categories learned from training.

---

***So in the above output we are getting this:***

    Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1228 stored elements and shape (614, 4)>

### What is a Sparse Matrix?

A sparse matrix is just a way to save space in memory.
Instead of storing all the 0s from One-Hot Encoding, it only stores the 1s and their positions.

### Why does this happen in One-Hot Encoding?

Example: suppose we encode Gender → Male / Female.

After One-Hot Encoding:

| Person | Male | Female |
| ------ | ---- | ------ |
| 1      | 1    | 0      |
| 2      | 0    | 1      |
| 3      | 1    | 0      |

Most of the numbers are 0.
Imagine if we had 100 categories → then every row would have 99 zeros and only 1 one.

Storing all those zeros would waste memory.

---

### There are two ways to turn the sparse matrix into a normal array (dense):

* **Method 1** → While creating the encoder

        ohe = OneHotEncoder(sparse_output=False)   # new versions use sparse_output=False
        encoded = ohe.fit_transform(data)

This directly gives you a NumPy array (with 0s and 1s).

* **Method 2** → After encoding (convert manually)

        ohe = OneHotEncoder()   # default gives sparse matrix
  
        encoded_sparse = ohe.fit_transform(data)
  
        # Convert sparse → dense (normal array)
        encoded_dense = encoded_sparse.toarray()

#### Both do the same thing.

* `sparse=False` → tells scikit-learn: “Don’t give me sparse, give me array directly.”

* `.toarray()` → you take the sparse matrix and convert it later.

#### Think of it like this:

* Sparse = a list of where the lights are ON (saves memory).

* Dense = a big grid showing every light, ON or OFF (takes more space).

In [26]:
# Using the method 1
ohe = OneHotEncoder(sparse_output=False)   # New versions use sparse_output=False
encoded = ohe.fit_transform(en_data)

In [27]:
encoded

array([[0., 1., 1., 0.],
       [0., 1., 0., 1.],
       [0., 1., 0., 1.],
       ...,
       [0., 1., 0., 1.],
       [0., 1., 0., 1.],
       [1., 0., 1., 0.]], shape=(614, 4))

But now I need the data not in array but in form of a dataframe.

So inorder to convert them into Dataframe we will use:

`pd.DataFrame()`

In [28]:
pd.DataFrame(encoded,columns=['Gender_Female','Gender_Male','Married_No','Married_Yes'])

,Gender_Female,Gender_Male,Married_No,Married_Yes
0,0.0,1.0,1.0,0.0
1,0.0,1.0,0.0,1.0
2,0.0,1.0,0.0,1.0
3,0.0,1.0,0.0,1.0
4,0.0,1.0,1.0,0.0
...,...,...,...,...
609,1.0,0.0,1.0,0.0
610,0.0,1.0,0.0,1.0
611,0.0,1.0,0.0,1.0
612,0.0,1.0,0.0,1.0


When we apply One-Hot Encoding, it creates a new column for each category.

For example, if a column has two categories like Male and Female, it will create two new columns: `Gender_Male` and `Gender_Female`.

But notice something:

* If `Gender_Male = 1`, then `Gender_Female` will always be `0`.

* If `Gender_Female = 1`, then `Gender_Male` will always be `0`.

This means one of the columns is redundant. To avoid this, we can drop one column using the argument:

        `OneHotEncoder(drop='first')`

This way, for two categories, only one column is created.
So instead of storing both `Male` and `Female`, the model only needs one column:

* `0` = Female

* `1` = Male

This reduces unnecessary columns while still keeping all the information

***In short: `drop='first'` helps reduce the number of columns and avoids redundancy, especially when you have many categories.***

Now, imagine the Gender column had 3 categories: Male, Female, and Other.
When using OneHotEncoder with `drop='first'`, the encoder will drop one category (in this case, "Male") and only create columns for the remaining two:

| Person | Gender_Female  | Gender_Other                 |
| ------ | -------------- | ---------------------------- |
| 1      | 0              | 0   → Male (the dropped one) |
| 2      | 1              | 0   → Female                 |
| 3      | 0              | 1   → Other                  |

So how do we interpret this?

* If both are 0 → Gender is Male (the dropped category).

* If Gender_Female = 1 → Gender is Female.

* If Gender_Other = 1 → Gender is Other.

This way, we still represent all categories without creating an extra column for every single one, which helps avoid redundancy.

In [29]:
ohe = OneHotEncoder(sparse_output=False,drop='first')
encoded = ohe.fit_transform(en_data)

In [30]:
pd.DataFrame(encoded,columns=['Gender_Male','Married_Yes'])

,Gender_Male,Married_Yes
0,1.0,0.0
1,1.0,1.0
2,1.0,1.0
3,1.0,1.0
4,1.0,0.0
...,...,...
609,0.0,0.0
610,1.0,1.0
611,1.0,1.0
612,1.0,1.0


## Label Encoding

Label Encoding is performed on nominal data.

#### What is Nominal Data?

Nominal data is a type of categorical data where the categories represent labels or names. These categories have no inherent order, ranking, or logical connection between them.

##### You can think of it as "just names".

* You cannot compare them (e.g. Male is not greater than Female, Red is not smaller than Blue).

* You cannot do math on them (e.g. adding or averaging categories makes no sense).

##### Examples of Nominal Data

1. Gender: Male, Female, Other

2. Colors: Red, Blue, Green, Yellow

3. Cities: Delhi, Paris, Tokyo, New York

4. Animals: Dog, Cat, Rabbit

5. Yes/No Questions: Yes, No

In [31]:
df = pd.DataFrame({'name':['wscube','cow','cat','dog','black']})
df

,name
0,wscube
1,cow
2,cat
3,dog
4,black


In [32]:
from sklearn.preprocessing import LabelEncoder

In [33]:
le = LabelEncoder()
df['le_name'] = le.fit_transform(df['name'])

In [34]:
df

,name,le_name
0,wscube,4
1,cow,2
2,cat,1
3,dog,3
4,black,0


So when you used LabelEncoder, it assigned an integer value to each unique category in column 'name'.

So the mapping looks like:

    black → 0  
    cat   → 1  
    cow   → 2  
    dog   → 3  
    wscube→ 4

#### What this means:

* Each category (string) is mapped to a unique integer.

* The numbers don’t carry any order or ranking — they’re just IDs.

#### But here’s the catch:
Even though the numbers don’t mean an order, many ML models (like linear regression, decision trees) might interpret them as ordinal 

(thinking `wscube (4)` > `dog (3)` > `cat (1)` etc.), which can mislead the model if the data is truly nominal.

And that's why for nominal data (like names, cities, colors), we usually prefer OneHotEncoding over LabelEncoding.

---
***But what if I want to train the model with some names but the model won't understand it cause it 
understand 1 and 0 so how or what should I do??***

1. If names are just IDs (like customer name in hotel registration):

* Don’t train the model on them (drop the column).
    Because “John” vs “Sarah” doesn’t mean anything about hotel booking behavior.

2. If names are categories with small set of unique values (like 10 movie titles, or 20 city names):

* Use OneHotEncoder (or LabelEncoder if tree-based models).
* Example:

        movies = ["Inception", "Avatar", "Titanic"]


* OneHot → 3 columns (Inception=1, Avatar=0, Titanic=0) etc.

3. If names are many and meaningful (like 10,000 unique products, hotel names, or words):

* OneHotEncoding will explode (too many columns). Instead:

* Frequency Encoding → Replace each name with how many times it appears.

* Example:

        Name   Count
        John   120
        Sarah   80
        Alex    50


##### Model sees numbers, not 0/1 columns.

* Target Encoding → Replace names with their average target value.
    Example: if we predict hotel bookings, encode hotel names with their average booking rate.

* Embeddings (NLP style) → For text-like names (like movie titles, reviews, product names), use word embeddings (Word2Vec, BERT, etc.)
    or Scikit-learn’s TfidfVectorizer.
    Example: “Hotel Sunshine” → vector of numbers like [0.12, 0.87, ...].

`✅ So, if names don’t mean anything → drop them.`

`✅ If names are categories with small values → OneHot or LabelEncode.`

`✅ If names carry semantic meaning (words, titles, products, hotels, cities) → use embeddings or frequency/target encoding.`

* Label Encoding → converts categories into numbers (but beware, the order might mislead some models).

* OneHot Encoding → makes binary columns for each category (safe, but creates too many columns if categories are many).

#### For now, think simple:

* If your categorical column has few unique values (like Gender, Yes/No, Country with 5 options) → use OneHotEncoder.

* If it has more unique values (like 100 names) → LabelEncoder can be used, but you must understand the risk that the model might think there’s an order.

In [35]:
# Set the path to the file you'd like to load
file_path = "loan.csv"  # file inside tanishaj225/loancsv dataset

# Load dataset into pandas DataFrame
da = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "tanishaj225/loancsv",   # dataset slug
    path=file_path           # specify file to load
)

In [36]:
da.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [37]:
da.Property_Area.isna().sum()

np.int64(0)

In [38]:
da.Property_Area.unique()

array(['Urban', 'Rural', 'Semiurban'], dtype=object)

---
By default, `LabelEncoder` in scikit-learn does not handle NaN values directly. If there’s a missing value (`NaN`) in the column,
it usually throws an error instead of assigning a number.

However, if you fill NaN with a placeholder (like "`Unknown`") before applying `LabelEncoder`, then it will assign the next 
number (e.g., 3 in our case).

#### Note: 

If the column contained a missing value (`NaN`), **LabelEncoder** would not handle it directly. But if we replace NaN with a placeholder (like ‘`Unknown`’), then it would be assigned the next available index (e.g., 3).

---

In [39]:
la = LabelEncoder()
la.fit(da.Property_Area)

LabelEncoder()

In [40]:
la.transform(da.Property_Area)

array([2, 0, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2,
       1, 0, 1, 1, 1, 2, 2, 1, 2, 2, 0, 1, 0, 2, 2, 1, 2, 1, 2, 2, 2, 1,
       2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 1, 1, 0, 2, 2, 2, 2, 0, 0, 1, 1,
       2, 2, 2, 1, 2, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1,
       2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 2, 1, 2, 2, 2, 0, 2, 1,
       2, 1, 0, 1, 1, 0, 1, 2, 0, 2, 0, 1, 1, 1, 0, 0, 0, 0, 2, 0, 2, 2,
       1, 1, 1, 1, 0, 2, 1, 0, 0, 2, 1, 1, 2, 1, 2, 2, 0, 1, 0, 0, 2, 0,
       2, 1, 0, 2, 0, 1, 1, 2, 1, 0, 2, 0, 0, 0, 1, 1, 0, 2, 0, 1, 1, 0,
       0, 1, 1, 2, 2, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 2, 1, 0, 1, 0, 2,
       1, 2, 1, 1, 2, 2, 1, 1, 2, 0, 2, 1, 1, 1, 2, 0, 2, 1, 0, 1, 1, 1,
       2, 1, 1, 1, 1, 0, 2, 1, 1, 0, 1, 0, 0, 1, 1, 0, 2, 2, 0, 1, 0, 2,
       2, 0, 1, 2, 2, 2, 1, 2, 1, 2, 0, 1, 2, 0, 0, 2, 0, 1, 2, 1, 1, 0,
       1, 0, 1, 2, 0, 2, 2, 2, 0, 1, 1, 1, 1, 2, 1, 0, 2, 1, 2, 2, 0, 0,
       1, 0, 1, 0, 0, 1, 2, 2, 1, 2, 1, 2, 0, 2, 2,

*As you can see, LabelEncoding has successfully converted the categories into numeric values. The numbers you see (0, 1, 2) are simply labels assigned to each category:*

0 → Rural

1 → Semiurban

2 → Urban

These numbers don’t represent any order or hierarchy — they are just identifiers for the categories. 

In [41]:
# We will write the data back into the dataset
da.Property_Area = la.transform(da.Property_Area)

In [42]:
da.Property_Area.unique()

array([2, 0, 1])

In [43]:
da.Property_Area

0      2
1      0
2      2
3      2
4      2
      ..
609    0
610    0
611    2
612    2
613    1
Name: Property_Area, Length: 614, dtype: int64

---
## Ordinal Encoding

Ordinal Encoding can be performed in two different ways

1. Using `OrdinalEncoder` from scikit-learn

This is the most common method. You can define the order of categories explicitly.

In [44]:
from sklearn.preprocessing import OrdinalEncoder

# Example data
data = pd.DataFrame({'Size': ['Small', 'Medium', 'Large', 'Medium', 'Small']})

# Define the order
encoder = OrdinalEncoder(categories=[['Small', 'Medium', 'Large']])
data['Size_encoded'] = encoder.fit_transform(data[['Size']])

print(data)

     Size  Size_encoded
0   Small           0.0
1  Medium           1.0
2   Large           2.0
3  Medium           1.0
4   Small           0.0


Here, the encoder respected the order `Small < Medium < Large`.

---

### Note:

By default, OrdinalEncoder(categories='auto') assigns numbers to categories based on the order they appear in the data, not alphabetically.

If you want alphabetical order (or any custom order), you must explicitly pass the list of categories in the order you want.

For example:

    import pandas as pd
    from sklearn.preprocessing import OrdinalEncoder

    data = pd.DataFrame({'City': ['Delhi', 'Mumbai', 'Chennai', 'Delhi', 'Bangalore']})

    encoder = OrdinalEncoder(categories='auto')
    data['City_encoded'] = encoder.fit_transform(data[['City']])

    print(data)

**OUTPUT**

    City  City_encoded
    0     Delhi          0
    1    Mumbai          1
    2   Chennai          2
    3     Delhi          0
    4  Bangalore          3


#### Notice:

It didn’t follow alphabetical order (alphabetically it should’ve been Bangalore=0, Chennai=1, Delhi=2, Mumbai=3).
Instead, it just followed the order in which categories appeared.

Therefor what I mean to say is:

`OrdinalEncoder` does not automatically order categories alphabetically.

Instead, it orders them in the order they appear in your data (first appearance = 0, next new category = 1, and so on).

---

2. Manual Mapping (Dictionary Mapping with `map()` or `replace()`)

Sometimes you don’t need sklearn — you can just map categories to numbers manually.

In [45]:
# Define a mapping
size_mapping = {'Small': 0, 'Medium': 1, 'Large': 2}

# Apply mapping
data['Size_encoded_map'] = data['Size'].map(size_mapping)

print(data)

     Size  Size_encoded  Size_encoded_map
0   Small           0.0                 0
1  Medium           1.0                 1
2   Large           2.0                 2
3  Medium           1.0                 1
4   Small           0.0                 0


### Summary:

* Method 1: `OrdinalEncoder` → More flexible, works well in pipelines.

* Method 2: `Manual mapping` → Simpler and quicker for small tasks.

---

You might just come up with a question like:
--

When I am transforming the categorical data into numerical data through encoding,

For example: 

    rural = 0, 
    semiurban = 1, 
    and urban = 2 

And I have also uploaded it inplace of the original column so here is my question:

Does the machine learning model understand that 0 is for rural and all and if it doesn't then how does it predict so accurately and if he does understand which number belongs to which one then how??

So the gpt replied:

1. **Does the ML model understand that 0 = Rural, 1 = Semiurban, 2 = Urban?**

     No. The model does not know the meaning of these numbers (like humans do).
     For the model, they are just numeric tokens — features that it can use in its math.

---

2. **Then how does the model still make good predictions?**

    It depends on which encoding method and which algorithm you’re using:

    `Case A`: Ordinal Encoding (0,1,2)


    * When you encode `Rural=0, Semiurban=1, Urban=2`, the model may assume order/relationship between categories (e.g., "Urban > Semiurban > Rural").

    * This is fine *if the categories are truly ordered* (like "Low, Medium, High").

    * But if categories are **nominal** (no ranking), this may mislead some algorithms (especially linear models).

    `Case B`: One-Hot Encoding

    * Here, categories are converted into independent columns like `is_Rural`, `is_Semiurban`, `is_Urban`.

    * The model treats them as **separate groups with no ordering**.

    * This is safer for **nominal data** because no "fake ordering" is introduced.

---

3. **How does the model make predictions then?**

    Think of it like this:

    * The model doesn’t know "Rural means village".

    * It only sees patterns in the data:

        * Maybe in training data, `Rural (0)` → usually leads to lower loan approval.

        * `Urban (2)` → usually leads to higher loan approval.

    * So the model learns the statistical relationship between the encoded numbers and the target variable.

👉 In short:
The model **doesn’t understand meaning**, it just **learns correlations** between the numbers and the output.

---

4. Example analogy:

Imagine I give you a secret code:

* 🍎 = 0, 🍌 = 1, 🍇 = 2.
    You don’t know what the numbers mean.
    But if I show you enough data like:

* 0 → eaten in the morning,

* 1 → eaten at night,

* 2 → used for juice,

You’ll learn the *patterns even* without knowing what fruit 🍎🍌🍇 really are.

✅ So, the **model doesn’t understand Rural/Urban like a human**, but it **learns the patterns from training data** to make accurate predictions.

---

And this is what I came up with
--

So ya, it's bassically a puzzle solving game but with numbers.

Machine learning is like giving the model a *giant puzzle* made of numbers.

* The model doesn’t know the *“story”* behind the numbers (Rural, Urban, Cat, Dog, etc.).

* It only knows: *“when I see these numbers together, usually this output happens.”*

* Over time (through training), it **learns the hidden rules of the puzzle** — just like you would learn that certain puzzle pieces always fit together.

##### That’s why encoding is so important:

* We’re basically **translating human labels (words/categories)** into **numbers** so the model can play its puzzle-solving game.

The better we translate (encode), the easier the model can “see” the hidden patterns.

#### Fun way to think about it:

Humans → understand the story.

Models → understand the math behind the story.

---

---

A fun little mini-puzzle analogy between Label Encoding vs OneHot so you’ll never forget it
--

### Puzzle 1: Label Encoding

Imagine you’re given 3 keys for 3 different doors:

* Rural = 🔑0

* Semiurban = 🔑1

* Urban = 🔑2

Now, the ML model doesn’t “know” these are cities. It just knows:
👉 If the puzzle input says 0, open door #0.
👉 If it says 1, open door #1.
👉 If it says 2, open door #2.

💡 The trick: The numbers **don’t mean bigger or better**. They’re just labels on keys. But some models (like Linear Regression) might accidentally think **2 > 0**, which could mess things up.

### Puzzle 2: OneHot Encoding

Now imagine instead of numbers, you’re given 3 switches:

* Rural = [1, 0, 0]

* Semiurban = [0, 1, 0]

* Urban = [0, 0, 1]

Here, the model doesn’t compare numbers. It just checks:
👉 Which switch is ON (1)?

💡 This way, **there’s no confusion about order**. Each category is independent.

✅ So the puzzle difference:

* `Label Encoding` = One key (number) per category.

* `OneHot Encoding` = One switch per category.